In [113]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.fit_manager as fm
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [114]:
from importlib import reload
reload(fm)
reload(dl)
reload(fit)

<module 'fitter.fitter' from '../fitter/fitter.py'>

### Title

In [115]:
params = {
    'collection_name' : 'test', 
    
    'model_info' : {
        'fit_type' : 'xpt', # 'ma', 'ma-ratio', 'xpt', 'xpt-ratio', or 'poly'
        'order' : 'n3lo', # 'nlo', 'n2lo', 'n3lo'
        'latt_ct' : 'n4lo',
        'F2' : 'FKFK', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)

        'include_FV' : True, # max 10
        'exclude' : [], # put LECs here
        'use_bijnens_central_value' : True,
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : False, 
        'include_log2' : False,
        'include_sunset' : False,
    }, 

    'fast_susnet' : True,
    'use_prior' : False,
    'bias_correct' : True,
    #'include_su2_isospin_corrrection' : False,
    
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
}

In [116]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()
params['model_info']['name'] = data_loader.get_model_name_from_model_info(params['model_info'])


# Make bootstrapper
fit_manager = fm.fit_manager(fit_data, phys_point_data, prior=None, **params)

new_prior = fit_manager.create_prior_from_fit()
#data_loader.save_prior(new_prior, bootstrapper.get_name())
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.
1   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
2   {'chiral_n2lo': 2.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
3   {'chiral_n2lo': 1.0,'chiral_n3lo': 2.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
4   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 101.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
5   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
6   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.0,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
7   {'chiral_n2lo': 0.01,'chiral_n3lo': 1.4,'latt_n2lo': 100.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
8   {'chiral_n2lo': 1.0,'chiral_n3lo': 1.6,'latt_n2lo': 101.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
9   {'chiral_n2lo': 0.6,'chiral_n3lo': 0.16,'latt_n2lo': 101.0,'latt_n3lo': 1000.0,'latt_n4lo': 1000.0}
10   {'chiral_n2lo': 0.44,'chiral_n3lo': 0.824,'la

79   {'chiral_n2lo': 1.3,'chiral_n3lo': 0.855,'latt_n2lo': 7.41,'latt_n3lo': 1030.0,'latt_n4lo': 1040.0}
80   {'chiral_n2lo': 1.1,'chiral_n3lo': 0.736,'latt_n2lo': 15.5,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
81   {'chiral_n2lo': 1.24,'chiral_n3lo': 0.747,'latt_n2lo': 8.22,'latt_n3lo': 1030.0,'latt_n4lo': 1040.0}
82   {'chiral_n2lo': 1.21,'chiral_n3lo': 0.756,'latt_n2lo': 10.4,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
83   {'chiral_n2lo': 1.28,'chiral_n3lo': 0.804,'latt_n2lo': 7.52,'latt_n3lo': 1030.0,'latt_n4lo': 1040.0}
84   {'chiral_n2lo': 1.15,'chiral_n3lo': 0.753,'latt_n2lo': 13.5,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
85   {'chiral_n2lo': 0.74,'chiral_n3lo': 1.47,'latt_n2lo': 11.7,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
86   {'chiral_n2lo': 0.613,'chiral_n3lo': 0.0406,'latt_n2lo': 11.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
87   {'chiral_n2lo': 0.233,'chiral_n3lo': 0.754,'latt_n2lo': 12.0,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
88   {'chiral_n2lo': 0.474,'chiral_n3lo': 0.31

155   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.0138,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
156   {'chiral_n2lo': 0.0191,'chiral_n3lo': 0.0274,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
157   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.01,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
158   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.0114,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
159   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.0211,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
160   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.01,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
161   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.0112,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
162   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.0199,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
163   {'chiral_n2lo': 0.01,'chiral_n3lo': 0.01,'latt_n2lo': 10.9,'latt_n3lo': 1020.0,'latt_n4lo': 1040.0}
164   {'chiral_n2lo': 0.01,'chir

In [117]:
print(fit_manager)


Model: xpt_FKFK_n3lo_a4_a6_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1924(34) [1.1957(19)]	(delta_su2 = -0.00415(70))

Parameters:
            L_4   -0.00224 (11)      [   0.0000 (50) ]  
            L_5   0.000414 (87)      [   0.0000 (50) ]  
            A_k    6(10000)e-06      [    0.000 (10) ]  
            A_p   -2(10000)e-06      [    0.000 (10) ]  
            A_a     -10.8 (1.4)      [        0 (11) ]  
           A_aa         79 (20)      [      0 (1020) ]  
           A_ak   -3(10000)e-06      [    0.000 (10) ]  
           A_ap   -8(10000)e-06      [    0.000 (10) ]  
           A_kk   3e-07 +- 0.01      [    0.000 (10) ]  
           A_kp   -3(10000)e-06      [    0.000 (10) ]  
           A_pp   -7(10000)e-06      [    0.000 (10) ]  
          A_aaa       -6 (1034)      [      0 (1040) ]  

Least Square Fit:
  chi2/dof [dof] = 0.78 [17]    Q = 0.72    logGBF = 68.147

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10)    (itns/time = 1*

In [108]:
print(fit_manager)


Model: xpt_FKFK_n3lo_a4_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1924(33) [1.1957(19)]	(delta_su2 = -0.00415(70))

Parameters:
            L_4    -0.00224 (11)       [   0.0000 (50) ]  
            L_5    0.000414 (88)       [   0.0000 (50) ]  
            A_k   0.00002 (1920)       [    0.000 (19) ]  
            A_p    -7(19200)e-06       [    0.000 (19) ]  
            A_a      -10.8 (1.4)       [        0 (11) ]  
           A_aa          79 (17)       [      0 (1080) ]  
           A_ak    -0.0003 (975)       [    0.000 (98) ]  
           A_ap    -0.0007 (975)       [    0.000 (98) ]  
           A_kk   0.00003 (9750)       [    0.000 (98) ]  
           A_kp    -0.0003 (975)       [    0.000 (98) ]  
           A_pp    -0.0006 (975)       [    0.000 (98) ]  

Least Square Fit:
  chi2/dof [dof] = 0.78 [17]    Q = 0.72    logGBF = 68.095

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 7/0.6)

Error Budget (relative error):


In [100]:
print(fit_manager)


Model: xpt_FKFK_n2lo_a4_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1924(33) [1.1957(19)]	(delta_su2 = -0.00415(70))

Parameters:
            L_4   -0.00224 (11)      [   0.0000 (50) ]  
            L_5   0.000414 (87)      [   0.0000 (50) ]  
            A_k    6(10000)e-06      [    0.000 (10) ]  
            A_p   -2(10000)e-06      [    0.000 (10) ]  
            A_a     -10.8 (1.4)      [        0 (11) ]  
           A_aa         79 (17)      [      0 (1050) ]  

Least Square Fit:
  chi2/dof [dof] = 0.78 [17]    Q = 0.72    logGBF = 68.124

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 8/0.6)

Error Budget (relative error):
  disc: 0.00031
  chiral: 4.68e-05
  pp_input: 0.00109
  stat: 0.00255



In [5]:
print(fit_manager)


Model: xpt_FKFK_n2lo_a4_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1926(38) [1.1957(19)]	(delta_su2 = -0.00416(70))

Parameters:
            L_4    -0.0018 (25)      [   0.0000 (50) ]  
            L_5    0.00036 (79)      [   0.0000 (50) ]  
            A_k       1.0 (4.6)      [     0.0 (5.5) ]  
            A_p       0.3 (2.8)      [     0.0 (5.5) ]  
            A_a     -10.9 (1.5)      [     0 (18000) ]  
           A_aa         81 (22)      [     0 (35600) ]  

Least Square Fit:
  chi2/dof [dof] = 0.72 [17]    Q = 0.79    logGBF = 56.725

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 14/0.7)

Error Budget (relative error):
  disc: 1.75e-06
  chiral: 0.000771
  pp_input: 0.00112
  stat: 0.0029



In [202]:
print(fit_manager)


Model: xpt_FKFK_n2lo_a4_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1926(38) [1.1957(19)]	(delta_su2 = -0.00416(70))

Parameters:
            L_4    -0.0018 (25)      [   0.0000 (50) ]  
            L_5    0.00036 (79)      [   0.0000 (50) ]  
            A_k       1.0 (4.6)      [     0.0 (5.5) ]  
            A_p       0.3 (2.8)      [     0.0 (5.5) ]  
            A_a     -10.9 (1.5)      [      0 (1000) ]  
           A_aa         80 (22)      [      0 (1000) ]  

Least Square Fit:
  chi2/dof [dof] = 0.72 [17]    Q = 0.79    logGBF = 63.182

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10)    (itns/time = 1*/0.3)

Error Budget (relative error):
  disc: 6.2e-05
  chiral: 0.00077
  pp_input: 0.00112
  stat: 0.0029



In [127]:
print(fit_manager)


Model: xpt_FKFK_n2lo_a4_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1927(33) [1.1957(19)]	(delta_su2 = -0.00416(70))

Parameters:
            L_4     -0.00224 (11)       [   0.0000 (50) ]  
            L_5     0.000422 (88)       [   0.0000 (50) ]  
            A_k    0.00001 (1562)       [    0.000 (16) ]  
            A_p   -9e-07 +- 0.016       [    0.000 (16) ]  
            A_a       -11.0 (1.4)       [       0 (100) ]  
           A_aa           81 (18)       [      0 (1000) ]  

Least Square Fit:
  chi2/dof [dof] = 0.72 [17]    Q = 0.78    logGBF = 66.457

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08,1e-10,1e-10)    (itns/time = 1*/0.2)

Error Budget (relative error):
  disc: 5.35e-05
  chiral: 4.8e-05
  pp_input: 0.00109
  stat: 0.00259



In [105]:
print(fit_manager.fmt_error_budget())

Partial Errors:
                  FK/Fpi
------------------------
          A_a:   0.00004
         A_aa:   0.00005
          A_k:   0.00001
          A_p:   0.00001
          L_4:   0.00003
          L_5:   0.00005
     phys:mpi:   0.00012
      phys:mk:   0.00021
phys:lam2_chi:   0.00128
         stat:   0.00308
------------------------
        total:   0.00335



In [19]:
new_prior

{'A_a': 0(100),
 'A_aa': 0(1000),
 'A_k': 0.0(1.0)e-05,
 'A_p': 0.0(1.0)e-05,
 'L_4': 0.0000(50),
 'L_5': 0.0000(50)}

In [18]:
print(fit_manager)


Model: xpt_FKFK_n2lo_a4_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.1927(33) [1.1957(19)]	(delta_su2 = -0.00416(70))

Parameters:
            L_4     -0.00224 (11)       [   0.0000 (50) ]  
            L_5     0.000422 (88)       [   0.0000 (50) ]  
            A_k    5e-12 +- 1e-05       [  0.0(1.0)e-05 ]  
            A_p   -4e-13 +- 1e-05       [  0.0(1.0)e-05 ]  
            A_a       -11.0 (1.4)       [       0 (100) ]  
           A_aa           81 (18)       [      0 (1000) ]  

Least Square Fit:
  chi2/dof [dof] = 0.72 [17]    Q = 0.78    logGBF = 66.457

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 5/0.3)

Error Budget (relative error):
	disc: 5.35e-05	chiral: 4.69e-05	pp_input: 0.00109	stat: 0.00259
